# SVD 알고리즘 기본 코드

행렬분해 기반 잠재 요인 협업 필터링

## #01. 준비작업

### [1] 패키지 가져오기

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Intel SKlearn 하드웨어 가속 패치 설정
import sys
if sys.platform == 'win32':
    from sklearnex import patch_sklearn
    patch_sklearn()

from helper.util import *
from helper.plot import *
from helper.analysis import *

from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from surprise.accuracy import rmse, mae

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### [2] 데이터 가져오기

In [2]:
origin = my_read_excel("https://data.hossam.kr/mldata/movie_ratings.xlsx", 
                    sheet_name='ratings', info=False)

## #02. 데이터 전처리

### [1] Surprise 형식의 데이터로 변환

`사용자 번호, 아이템 번호, 평점` 구조의 데이터를 만족해야 한다.

In [3]:
df = origin.drop('timestamp', axis=1)

# 평점의 분포를 알려준다.
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(df, reader)
data

## #03. 추천 모형 구현

### [1] 하이퍼파라미터 튜닝

In [4]:
params = {
    "n_epochs": [20, 40, 50],          # 반복 횟수(기본값=20)
    "n_factors": [100, 200, 300],      # 요인의 수(기본값=100)
}

# RandomizedSearchCV가 에러가 있는 듯 (아니면 파라미터 설정 방법이 다르거나...)
grid = GridSearchCV(SVD, 
                    param_grid=params, 
                    measures=['RMSE', 'MAE'], 
                    cv=5, 
                    n_jobs=-1)

grid.fit(data)

### [2] 결과확인

#### (1) 하이퍼파라미터와 성능 평가 지표 확인

In [5]:
grid.cv_results['params']

[{'n_epochs': 20, 'n_factors': 100},
 {'n_epochs': 20, 'n_factors': 200},
 {'n_epochs': 20, 'n_factors': 300},
 {'n_epochs': 40, 'n_factors': 100},
 {'n_epochs': 40, 'n_factors': 200},
 {'n_epochs': 40, 'n_factors': 300},
 {'n_epochs': 50, 'n_factors': 100},
 {'n_epochs': 50, 'n_factors': 200},
 {'n_epochs': 50, 'n_factors': 300}]

In [6]:
df = DataFrame(grid.cv_results['params'])
df['rmse'] = grid.best_score['rmse']
df['mae'] = grid.best_score['mae']
df.sort_values('rmse', ascending=False, inplace=True)
my_pretty_table(df)

+----+------------+-------------+----------+----------+
|    |   n_epochs |   n_factors |     rmse |      mae |
|----+------------+-------------+----------+----------|
|  0 |         20 |         100 | 0.873711 | 0.671291 |
|  1 |         20 |         200 | 0.873711 | 0.671291 |
|  2 |         20 |         300 | 0.873711 | 0.671291 |
|  3 |         40 |         100 | 0.873711 | 0.671291 |
|  4 |         40 |         200 | 0.873711 | 0.671291 |
|  5 |         40 |         300 | 0.873711 | 0.671291 |
|  6 |         50 |         100 | 0.873711 | 0.671291 |
|  7 |         50 |         200 | 0.873711 | 0.671291 |
|  8 |         50 |         300 | 0.873711 | 0.671291 |
+----+------------+-------------+----------+----------+


#### (3) 최적 하이퍼파라미터 확인

In [7]:
grid.best_params

{'rmse': {'n_epochs': 20, 'n_factors': 100},
 'mae': {'n_epochs': 20, 'n_factors': 100}}

#### (4) 최적 추정기

In [8]:
grid.best_estimator

{'rmse': <surprise.prediction_algorithms.matrix_factorization.SVD at 0x1977840dad0>,
 'mae': <surprise.prediction_algorithms.matrix_factorization.SVD at 0x1977ec8c310>}

#### (5) 최적 추정기를 활용한 컨텐츠 추천

학습을 다시 시켜야 한다.

In [9]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

In [10]:
estimator = grid.best_estimator['rmse']
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

[Prediction(uid=603, iid=3996, r_ui=5.0, est=4.119850407254607, details={'was_impossible': False}),
 Prediction(uid=199, iid=2912, r_ui=4.0, est=3.411465364444666, details={'was_impossible': False}),
 Prediction(uid=416, iid=2716, r_ui=2.0, est=3.2481094852920194, details={'was_impossible': False}),
 Prediction(uid=589, iid=150, r_ui=4.0, est=4.351755570541779, details={'was_impossible': False}),
 Prediction(uid=307, iid=6755, r_ui=4.0, est=2.461231812108919, details={'was_impossible': False})]

In [11]:
estimator.predict(uid=603, iid=3996)

Prediction(uid=603, iid=3996, r_ui=None, est=4.119850407254607, details={'was_impossible': False})